In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf

In [2]:
dataFrame = pd.read_table(r"../Data/dataNormalisee(14).csv", sep = ";", engine = 'python')

In [3]:
dataFrame_daily = pd.read_table(r"../Data/DailyData_VolumeProfile.csv", sep = ";", engine = 'python')

In [6]:
days = list(dataFrame_daily.Date)
daily_open = list(dataFrame_daily['Open'])

In [16]:
# pour chaque jour, on fait une requete pour selectionner la ligne ou l'heure la derniere de la journee
# permet de recuperer la liste des fermtures journalieres

daily_last = []
for i in range(len(days)): 
    df_last_minute = dataFrame.loc[(dataFrame['Time'] == ' 09:00:00.0') & (dataFrame.Date == days[i])]
    if len(df_last_minute) == 0:
        df_last_minute = dataFrame.loc[(dataFrame['Time'] == ' 09:35:00.0') & (dataFrame.Date == days[i])]
    if len(df_last_minute) == 0:
        df_last_minute = dataFrame.loc[(dataFrame['Time'] == ' 13:45:00.0') & (dataFrame.Date == days[i])]

    cur_last = list(df_last_minute['Last'])
    if len(cur_last) > 0:
        daily_last.append(int(cur_last[0]))

In [17]:
len(daily_last)

2249

In [ ]:
# récupère les high (max), low (min), voulme bid & asc, number of trades (sum)
# on fait des requetes pour recuperer toutes les donnees d'une journee
# cela permet de calculer le high de la journee, le low de la journee et le volume total de la journee
# on repete cela pour tous les jours
# relativement long en temps d'exécution mais permet de s'assurer de prendre toutes les entrees d'une journee
# par ex, si une journee se termine a 22h05 et non 22h, cette technique permet de s'assurer de bien prendre toutes les tranches
heures = [' 08:00:00.0', ' 08:05:00.0', ' 08:10:00.0', ' 08:15:00.0', ' 08:20:00.0', ' 08:25:00.0', ' 08:30:00.0', ' 08:35:00.0', ' 08:40:00.0', ' 08:45:00.0', ' 08:50:00.0', ' 08:55:00.0', ' 09:00:00.0']
daily_high = []
daily_low = []
daily_volume = []
daily_nb_trades = []
daily_bidVolume =[]
daily_askVolume =[]
daily_jour=[]

for day in days:
    df_day = dataFrame.loc[(dataFrame.Date == day) & (dataFrame[' Time'].isin(heures))]
    if len(df_day) == 0: #cette journee commence apres l'IB
        df_jour_i = dataFrame_daily.loc[dataFrame_daily.Date == day]
        daily_volume.append(float(df_jour_i['Volume']))
        daily_high.append(float(df_jour_i['High']))
        daily_low.append(float(df_jour_i['Low']))
        daily_bidVolume.append(float(df_jour_i['Bid Volume']))
        daily_askVolume.append(float(df_jour_i['Ask Volume']))
    else:
        volume_journalier = np.sum(list(df_day[' Volume']))
        daily_volume.append(volume_journalier)

        high_journalier = max(list(df_day[' High']))
        daily_high.append(high_journalier)

        low_journalier = min(list(df_day[' Low']))
        daily_low.append(low_journalier)

        bid_volume_journalier = np.sum(list(df_day[' Bid Volume']))
        daily_bidVolume.append(bid_volume_journalier)

        ask_volume_journalier = np.sum(list(df_day[' Ask Volume']))
        daily_askVolume.append(ask_volume_journalier)

In [ ]:
daily_spread = [float(daily_high[i]-daily_low[i]) for i in range(len(daily_high))]
daily_delta = [float(daily_askVolume[i]) - float(daily_bidVolume[i]) for i in range(len(daily_askVolume))]

daily_cumulativeDelta = [daily_delta[0]]
for i in range(1, len(daily_delta)):
    nouvelle_valeur = daily_cumulativeDelta[-1] + daily_delta[i]
    daily_cumulativeDelta.append(nouvelle_valeur)

In [ ]:
def moyenne_mobile(k, liste):
    if k<23:
        sum_= 0
        for i in range(k+1):
            sum_ += liste[i]
        return sum_/(k+1)
    else:
        sum_ = 0
        for i in range(k, k-23, -1):
            sum_ += liste[i]
        return sum_/23

In [ ]:
daily_moy_mobile = [moyenne_mobile(k, daily_last) for k in range(len(daily_last))]

In [ ]:
weekdays = dataFrame_daily.Jour

In [ ]:
dataFrame_daily = pd.DataFrame()
dataFrame_daily.insert(loc = 0, column='Date', value=pd.Series(days))
dataFrame_daily.insert(loc = 1, column='Jour',  value=pd.Series(weekdays))
dataFrame_daily.insert(loc = 2, column='Open', value=pd.Series(daily_open))
dataFrame_daily.insert(loc = 3, column='High', value=pd.Series(daily_high))
dataFrame_daily.insert(loc = 4, column='Low', value=pd.Series(daily_low))
dataFrame_daily.insert(loc = 5, column='Last', value=pd.Series(daily_last))
dataFrame_daily.insert(loc = 6, column='Volume', value=pd.Series(daily_volume))
dataFrame_daily.insert(loc = 7, column='BidVolume', value=pd.Series(daily_bidVolume))
dataFrame_daily.insert(loc = 8, column='AskVolume', value=pd.Series(daily_askVolume))
dataFrame_daily.insert(loc = 9, column='Spread', value=pd.Series(daily_spread))
dataFrame_daily.insert(loc = 10, column='Delta', value=pd.Series(daily_delta))
dataFrame_daily.insert(loc = 11, column='CumulativeDelta', value=pd.Series(daily_cumulativeDelta))
dataFrame_daily.insert(loc = 12, column='MoyennneMobile', value=pd.Series(daily_moy_mobile))

In [ ]:
dataFrame_daily[76:81]

In [39]:
dataFrame_daily.to_csv('../Data/dataIB.csv', sep = ';', index=False)